In [2]:
#!pip install transformers

2023-10-18 06:29:47.444752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 06:29:48.946376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 95.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [5]:
import re, json, os, itertools
import numpy as np
import pandas as pd
#from transformers import AutoTokenizer, AutoModelForTokenClassification
#from transformers import pipeline

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
#model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [14]:
#kenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
df = pd.read_json ('projectdataset.jsonl', lines=True)
#df2 = pd.read_json ('/gdrive/My Drive/project/mytest_project_dataset.jsonl', lines=True)

In [15]:
df=df.drop(['meta','annotation_approver','comments'],axis=1)
df

,id,text,labels
0,15501,Exhibit 99.1 JOINT FILING AGREEMENT The unders...,"[[13, 35, DOC_NAME], [489, 505, AGMT_DATE], [5..."
1,15502,"[LOGO] AMENDMENT TO SECTION 2, PART B OF THE C...","[[7, 66, DOC_NAME], [166, 182, AGMT_DATE], [19..."
2,15503,Exhibit 10.1 ENDORSEMENT AGREEMENT THIS ENDORS...,"[[13, 34, DOC_NAME], [40, 61, DOC_NAME], [111,..."
3,15504,Exhibit 10.6 TRADEMARK LICENSE AGREEMENT This ...,[]
4,15505,CONFIDENTIAL PORTIONS OMITTED EXHIBIT 10.16 [L...,"[[78, 99, DOC_NAME], [105, 126, DOC_NAME], [18..."
...,...,...,...
344,15845,STRATEGIC ALLIANCE AGREEMENT This Strategic Al...,"[[0, 28, DOC_NAME], [34, 62, DOC_NAME], [113, ..."
345,15846,Exhibit 10.41 SPONSORSHIP AGREEMENT This Spons...,"[[14, 35, DOC_NAME], [41, 62, DOC_NAME], [107,..."
346,15847,EXHIBIT 10.43 Dated 29/3/18 Distributorship ag...,"[[20, 27, AGMT_DATE], [28, 53, DOC_NAME], [62,..."
347,15848,CONTENT DISTRIBUTION AND LICENSE AGREEMENT Dea...,[]


In [16]:
df_cut=df[df['labels'].apply(lambda x:len(x)>0)]

In [18]:
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 0 to 348
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      314 non-null    int64 
 1   text    314 non-null    object
 2   labels  314 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.8+ KB


In [19]:
#nlp=pipeline('ner',model=model,tokenizer=tokenizer)

In [20]:

df_cut['token']=df_cut['text'].apply(lambda x:nlp(x))

<ipython-input-20-1f7b6b45023e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['token']=df_cut['text'].apply(lambda x:nlp(x))


In [21]:
row = df_cut.iloc[4]
doc = row['token']
for start, end, label in row['labels']:
    print(start, end, label)
print(doc)

12 28 DOC_NAME
37 58 PARTY
63 101 PARTY
125 138 AGMT_DATE
148 169 PARTY
380 418 PARTY
Exhibit 4.5 SUPPLY AGREEMENT between PROFOUND MEDICAL INC. and PHILIPS MEDICAL SYSTEMS NEDERLAND B.V. THIS AGREEMENT is made July 31, 2017 BETWEEN: PROFOUND MEDICAL INC., a company incorporated under the laws of the province of Ontario and having its registered address at 2400 Skymark, Unit 6, Mississauga, Ontario L4W 5K5, Canada (hereinafter referred to as “Customer”) - and - PHILIPS MEDICAL SYSTEMS NEDERLAND B.V., a company incorporated under the laws of the Netherlands with its principal place of business at Veenpluis 4-6 5684 PC Best, the Netherlands (hereinafter referred to as “Philips”) Customer and Philips hereinafter also collectively referred to as the “Parties” and individually as a “Party”. WHEREAS: A. Pursuant to the Asset and Share Purchase Agreement (the “Purchase Agreement”) entered into on June 30, 2017 by Customer, Koninklijke Philips NV (“Philips NV”) N.V. and Customer agreed to exec

In [23]:
'''doc_name_count=0
party_count=0
date_count=0
for index,row in df_cut.iterrows():
    for l in row['labels']:
        if l[2]==row'''

"doc_name_count=0\nparty_count=0\ndate_count=0\nfor index,row in df_cut.iterrows():\n    for l in row['labels']:\n        if l[2]==row"

In [24]:
tags_list_iob = []
for index, row in df_cut.iterrows():
    doc = row['token']
    ents=[]
    for start, end, label in row['labels']:
        if doc.char_span(start, end, label) != None:
            ent = doc.char_span(start, end, label)
            ents.append(ent)
        elif doc.char_span(start, end+1, label) != None:
            ent = doc.char_span(start, end+1, label)
            ents.append(ent)
        elif doc.char_span(start+1, end, label) != None:
            ent = doc.char_span(start+1, end, label)
            ents.append(ent)
        elif doc.char_span(start, end-1, label) != None:
            ent = doc.char_span(start, end-1, label)
            ents.append(ent)
        elif doc.char_span(start-1, end, label) != None:
            ent = doc.char_span(start-1, end, label)
            ents.append(ent)
    doc.ents = ents
    iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ != "O" else "O" for t in doc]
    tags_list_iob.append(iob_tags)
df_cut['tags'] = tags_list_iob

<ipython-input-24-1d3424116e9a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['tags'] = tags_list_iob


In [25]:
df_cut

,id,text,labels,token,tags
0,15501,Exhibit 99.1 JOINT FILING AGREEMENT The unders...,"[[13, 35, DOC_NAME], [489, 505, AGMT_DATE], [5...","(Exhibit, 99.1, JOINT, FILING, AGREEMENT, The,...","[O, O, B-DOC_NAME, I-DOC_NAME, I-DOC_NAME, O, ..."
1,15502,"[LOGO] AMENDMENT TO SECTION 2, PART B OF THE C...","[[7, 66, DOC_NAME], [166, 182, AGMT_DATE], [19...","([, LOGO, ], AMENDMENT, TO, SECTION, 2, ,, PAR...","[O, O, O, B-DOC_NAME, I-DOC_NAME, I-DOC_NAME, ..."
2,15503,Exhibit 10.1 ENDORSEMENT AGREEMENT THIS ENDORS...,"[[13, 34, DOC_NAME], [40, 61, DOC_NAME], [111,...","(Exhibit, 10.1, ENDORSEMENT, AGREEMENT, THIS, ...","[O, O, B-DOC_NAME, I-DOC_NAME, O, B-DOC_NAME, ..."
4,15505,CONFIDENTIAL PORTIONS OMITTED EXHIBIT 10.16 [L...,"[[78, 99, DOC_NAME], [105, 126, DOC_NAME], [18...","(CONFIDENTIAL, PORTIONS, OMITTED, EXHIBIT, 10....","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-DOC_..."
5,15506,Exhibit 4.5 SUPPLY AGREEMENT between PROFOUND ...,"[[12, 28, DOC_NAME], [37, 58, PARTY], [63, 101...","(Exhibit, 4.5, SUPPLY, AGREEMENT, between, PRO...","[O, O, B-DOC_NAME, I-DOC_NAME, O, B-PARTY, I-P..."
...,...,...,...,...,...
343,15844,Promotion Agreement Between PageMaster Corpora...,"[[0, 19, DOC_NAME], [28, 50, PARTY], [55, 68, ...","(Promotion, Agreement, Between, PageMaster, Co...","[B-DOC_NAME, I-DOC_NAME, O, B-PARTY, I-PARTY, ..."
344,15845,STRATEGIC ALLIANCE AGREEMENT This Strategic Al...,"[[0, 28, DOC_NAME], [34, 62, DOC_NAME], [113, ...","(STRATEGIC, ALLIANCE, AGREEMENT, This, Strateg...","[B-DOC_NAME, I-DOC_NAME, I-DOC_NAME, O, B-DOC_..."
345,15846,Exhibit 10.41 SPONSORSHIP AGREEMENT This Spons...,"[[14, 35, DOC_NAME], [41, 62, DOC_NAME], [107,...","(Exhibit, 10.41, SPONSORSHIP, AGREEMENT, This,...","[O, O, B-DOC_NAME, I-DOC_NAME, O, B-DOC_NAME, ..."
346,15847,EXHIBIT 10.43 Dated 29/3/18 Distributorship ag...,"[[20, 27, AGMT_DATE], [28, 53, DOC_NAME], [62,...","(EXHIBIT, 10.43, Dated, 29/3/18, Distributorsh...","[O, O, O, B-AGMT_DATE, B-DOC_NAME, I-DOC_NAME,..."


In [26]:
all_tags = list(itertools.chain.from_iterable(tags_list_iob))

def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    unique_list.sort()
    return unique_list

feature_class_labels = unique(all_tags)
print(feature_class_labels)

['B-AGMT_DATE', 'B-DOC_NAME', 'B-PARTY', 'I-AGMT_DATE', 'I-DOC_NAME', 'I-PARTY', 'O']


In [31]:
with open('Feature_class_labels.json','w') as f:
   json.dump(feature_class_labels,f,indent=3)


In [32]:
df_cut['splited_token']=df_cut['token'].apply(lambda y:[tex.text for tex in y])
df_cut["ner_tags"]=df_cut['tags'].apply(lambda x:[feature_class_labels.index(tag) for tag in x])

<ipython-input-32-4de745ea7f15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['splited_token']=df_cut['token'].apply(lambda y:[tex.text for tex in y])
<ipython-input-32-4de745ea7f15>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["ner_tags"]=df_cut['tags'].apply(lambda x:[feature_class_labels.index(tag) for tag in x])


In [33]:
data=df_cut[['id','ner_tags','splited_token']]

In [38]:
with open('Data_ner_n.json','w') as f:
    data.to_json(f )